# Spatial Resolution & Swath vs Focal Length Trade
Tradeoff between spatial resolution and swath width for a given range of foreoptic focal lengths.

In [1]:
# stdlib
from pathlib import Path

# external
import astropy.units as unit
import numpy as np
import pandas as pd
import plot
from IPython.display import display

# project
from architect.systems.optical.spectrometers import HyperspectralImager
from architect.systems.optical.sensors import TauSWIR
from architect.systems.optical.foreoptics import Foreoptic
from architect.systems.optical.masks import RectSlit
from architect.libs import utillib

## Setup

In [2]:
output_dir = Path("output/spatial_resolution_&_swath_vs_focal_length_trade/")
output_dir.mkdir(parents=True, exist_ok=True)

### Parameters

In [4]:
# constants
orbital_altitude = 550 * unit.km
foreoptic_diameter = 44 * unit.mm
skew_angle = np.array([0, 0]) * unit.deg
slit_size = np.array([3, 1]) * unit.mm
target_wavelength = 1300 * unit.nm

# variables
focal_length = np.arange(start=10, stop=300, step=10) * unit.mm

# vectorization
parameter_space_shape = (focal_length.size, slit_size.size)
focal_length = utillib.hypercast(focal_length, slit_size)

""" focal_length = utillib.orient_and_broadcast(
    a=focal_length, dim=0, shape=parameter_space_shape
) """

' focal_length = utillib.orient_and_broadcast(\n    a=focal_length, dim=0, shape=parameter_space_shape\n) '

### Instantiation

In [5]:
# components
sensor = TauSWIR()

foreoptic = Foreoptic(
    diameter=foreoptic_diameter, focal_length=focal_length
)

slit = RectSlit(size=slit_size)

# systems
payload = HyperspectralImager(
    sensor=sensor, foreoptic=foreoptic, slit=slit
)

## Pipeline

In [6]:
sensor_spatial_resolution = payload.get_spatial_resolution(
    wavelength=target_wavelength,
    target_distance=orbital_altitude,
    skew_angle=skew_angle[0],
).to(unit.m)

display(
    f"Sensor-limited spatial resolution test shape: {sensor_spatial_resolution.shape}"
)

swath = payload.get_swath(altitude=orbital_altitude, skew_angle=skew_angle)
display(f"Swath shape: {swath.shape}")

UnitConversionError: Can only apply 'maximum' function to quantities with compatible dimensions

## Plots

In [7]:
focal_length_label = f"Focal Length [{focal_length.unit}]"
sensor_spatial_resolution_label = (
    f"Spatial Resolution [{sensor_spatial_resolution.unit}]"
)
swath_label_x = f"Swath (across-track) [{swath.unit}]"
swath_label_y = f"Swath (along-track) [{swath.unit}]"


data = {
    focal_length_label: focal_length[:, 0],
    sensor_spatial_resolution_label: sensor_spatial_resolution[:, 0],
    swath_label_x: swath[:, 0],
    swath_label_y: swath[:, 1],
}

df = pd.DataFrame.from_dict(data=data)
fig = plot.line(
    df=df,
    x=focal_length_label,
    y=[
        sensor_spatial_resolution_label,
        swath_label_x,
        swath_label_y,
    ],
    title=None,
    dark=True,
)
df.to_csv(output_dir / "data.csv", index=False)
plot.save(
    fig=fig, name="spatial_resolution_&_swath_vs_focal_length_trade", path=output_dir
)


fig.show()
display(df)
display(payload)

AttributeError: 'list' object has no attribute 'unit'